Import and setup some auxiliary functions

In [1]:
# Don't edit this cell
import torch
from torchvision import transforms, datasets
import numpy as np
import timeit
from collections import OrderedDict
from pprint import pformat
from torch.utils.data.sampler import *
from tqdm import tqdm
import time
from prettytable import PrettyTable

import torch.nn as nn
import torch.nn.functional as F

torch.multiprocessing.set_sharing_strategy('file_system')

def compute_score(acc, min_thres, max_thres):
    if acc <= min_thres:
        base_score = 0.0
    elif acc >= max_thres:
        base_score = 100.0
    else:
        base_score = float(acc - min_thres) / (max_thres - min_thres) \
                     * 100
    return base_score


def run(algorithm, dataset_name, filename):
    predicted_test_labels, gt_labels, run_time, parameters_count = algorithm(dataset_name)
    if predicted_test_labels is None or gt_labels is None:
      return (0, 0, 0, 0)

    correct = 0
    total = 0
    for label, prediction in zip(gt_labels, predicted_test_labels):
      total += label.size(0)
      correct += (prediction.cpu().numpy() == label.cpu().numpy()).sum().item()   # assuming your model runs on GPU
      
    accuracy = float(correct) / total
    
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
    return (correct, accuracy, run_time, parameters_count)

In [2]:
def load_data(dataset_name, device, config):
    """
    loads cifar-10 dataset using torchvision, take the last 5k of the training data to be validation data
    """

    training = datasets.CIFAR10(root='./data', train=True, download=True, transform=config['transforms'])
    test = datasets.CIFAR10(root='./data', train=False, download=True, transform=config['transforms'])
    
    trainingLoader = torch.utils.data.DataLoader(training,
                                                 batch_size=config['batch_size'],
                                                 sampler=SubsetRandomSampler(list(range(len(training) - 5000))),
                                                 num_workers=2)
    validationLoader = torch.utils.data.DataLoader(training,
                                                   batch_size=config['batch_size'],
                                                   sampler=SubsetRandomSampler(list(range(len(training) - 5000, len(training)))),
                                                   num_workers=2)
    testLoader = torch.utils.data.DataLoader(test, num_workers=2)

    return trainingLoader, validationLoader, testLoader

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(3,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.ConvTranspose2d(32,64,3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(1024,10)
        )

        # He Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m,nn.Linear):
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
                m.bias.detach().zero_()
                
    def forward(self, x):
        return self.seq(x)


In [4]:
def train(train_dataloader, valid_dataloader, device, config):
  net = Net().to(device)
  opt = torch.optim.Adam(net.parameters(), config['lr'], weight_decay=config['regular_constant'])

  # print(sum(p.numel() for p in net.parameters() if p.requires_grad))
  def validate():
    net.eval()
    loss = correct = 0
    
    with torch.no_grad():
      for data, target in valid_dataloader:
        data = data.to(device)
        target = target.to(device)

        output = net(data)
        pred = output.data.max(1, keepdim=True)[1]

        loss += F.cross_entropy(output, target, reduction='sum').item()
        correct += pred.eq(target.data.view_as(pred)).sum()

    loss /= len(valid_dataloader.dataset)

    print(f'Validation Set: AVG LOSS: {loss:.4f} ACC: {correct}/{len(valid_dataloader.dataset)} {(100. * correct / len(valid_dataloader.dataset)):.2f}%')

  def _train():
    net.train()

    bar = tqdm(train_dataloader, ncols=100, position=0, leave=True)
    avgLoss = 0

    for i, (data, target) in enumerate(bar):
      data = data.to(device)
      target = target.to(device)
      opt.zero_grad()

      output = net(data)
      
      loss = F.cross_entropy(output, target)
      avgLoss += (loss.item() - avgLoss) / (i+1)
      loss.backward()
      opt.step()

      bar.set_description(f'Train Loss: {loss.item():.6f} AVG Loss: {avgLoss:.6f}')

  # validate()
  for _ in range(config['num_epochs']):
    _train()

  return net

In [7]:
def test(model, test_dataloader, device):
  test_predictions = []
  true_labels = []

  model.eval()

  for data, target in test_dataloader:
    data = data.to(device)
    target = target.to(device)

    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]

    test_predictions.append(pred)
    true_labels.append(target)


  return test_predictions, true_labels

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    # uncomment below codes for your debugging
    # print(table)
    # print(f"Total Trainable Params: {total_params}")
    return total_params

def run_NN(dataset_name):
    # set parameters cifar10
  config = {
        'lr': 0.001,
        'num_epochs': 10,
        'batch_size': 128,
        'num_classes': 10,
        'regular_constant': 0.001,
        'transforms': transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) }
    
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  train_dataloader, valid_dataloader, test_dataloader = load_data(dataset_name, device, config)
  
  model = train(train_dataloader, valid_dataloader, device, config)
  parameters_count = count_parameters(model)

  device = torch.device("cpu")
  start_time = timeit.default_timer()
  assert test_dataloader.batch_size == 1, 'Error: You should use use batch size = 1 for the test loader.'
  preds, labels = test(model.to(device), test_dataloader, device)
  end_time = timeit.default_timer()
  

  test_time = (end_time - start_time)
  print("Total run time of testing the model: ", test_time , " seconds.")
    
  return preds, labels, test_time, parameters_count

Main loop. Run time and total score will be shown below.

In [ ]:
# Don't edit this cell
def run_on_dataset(dataset_name, filename):
    min_thres = 0.55
    max_thres = 0.65

    correct_predict, accuracy, run_time, parameters_count = run(run_NN, dataset_name, filename)

    score = compute_score(accuracy, min_thres, max_thres)
    if parameters_count > 50000:
      score = max(0, score - 25)
    result = OrderedDict(
                  score=score,
                  correct_predict=correct_predict,
                  accuracy=accuracy,
                  run_time=run_time,
                  parameters_count=parameters_count)
    return result, score


def main():
    filenames = { "CIFAR10": "predictions_cifar10_YoussefIsmail_1616494.txt"}
    result_all = OrderedDict()
    scores = []
    for dataset_name in ["CIFAR10"]:
        result_all, this_score = run_on_dataset(dataset_name, filenames[dataset_name])
    with open('result.txt', 'w') as f:
        f.writelines(pformat(result_all, indent=4))
    print("\nResult:\n", pformat(result_all, indent=4))


main()

Files already downloaded and verified
Files already downloaded and verified


Train Loss: 0.914007 AVG Loss: 0.783570: 100%|████████████████████| 352/352 [00:49<00:00,  7.09it/s]
